In [11]:
import json
import openai
import os
from dotenv import load_dotenv
from colorama import Fore, Style, init

load_dotenv()

True

In [12]:
client = openai.Client(api_key=os.getenv("OPENAI_API_KEY"))

init(autoreset=True)

In [13]:
def saudacao_por_periodo(horas):
    if 5 <= horas < 12:
        return json.dumps({"saudacao": "Bom dia!"})
    elif 12 <= horas < 18:
        return json.dumps({"saudacao": "Boa tarde!"})
    elif 18 <= horas < 22:
        return json.dumps({"saudacao": "Boa noite!"})
    else:
        return json.dumps({"saudacao": "Boa madrugada!"})

In [14]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "saudacao_por_periodo",
            "description": "Retorna uma saudação baseada no período do dia.",
            "parameters": {
                "type": "object",
                "properties": {
                    "horas": {
                        "type": "integer",
                        "description": "Hora atual em formato 24 horas (0-23)."
                    },
                },
                "required": ["horas"]
            }
        }
    }
]

In [23]:
funcao_disponivel = {
    "saudacao_por_periodo": saudacao_por_periodo   
}

In [32]:
mensagens = [{"role": "user", "content": "Qual saudação o modelo me dá se for 10h?"}]

In [33]:
resposta = client.chat.completions.create(
    model="gpt-3.5-turbo-0125",
    messages=mensagens,
    tools=tools,
    tool_choice="auto"
)

In [34]:
mensagem_resp = resposta.choices[0].message
mensagem_resp

ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_oVYXCCQol7kBuAjAppMvl9RP', function=Function(arguments='{"horas":10}', name='saudacao_por_periodo'), type='function')])

In [35]:
tool_calls = mensagem_resp.tool_calls
tool_calls

[ChatCompletionMessageToolCall(id='call_oVYXCCQol7kBuAjAppMvl9RP', function=Function(arguments='{"horas":10}', name='saudacao_por_periodo'), type='function')]

In [36]:
if tool_calls:
    mensagens.append(mensagem_resp)

    for tool_call in tool_calls:
        function_name = tool_call.function.name
        function_to_call = funcao_disponivel[function_name]
        function_args = json.loads(tool_call.function.arguments)
        function_response = function_to_call(
            horas = function_args.get("horas")
        )

        mensagens.append(
            {
               "tool_call_id": tool_call.id,
               "role": "tool",
               "name": function_name,
               "content": function_response
            }
       )

    
    segunda_resposta = client.chat.completions.create(
        model="gpt-3.5-turbo-0125",
        messages = mensagens
    )

In [37]:
mensagem_resposta = segunda_resposta.choices[0].message
mensagem_resposta

ChatCompletionMessage(content='Se forem 10h, o modelo me dará a saudação "Bom dia!".', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None)